# 如何流式传输聊天模型响应


所有[聊天模型](https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.chat_models.BaseChatModel.html)都实现了[Runnable 接口](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)，该接口附带了标准可运行方法（即`ainvoke`、`batch`、`abatch`、`stream`、`astream`、`astream_events`）的**默认**实现。

**默认**流式传输实现提供了一个`Iterator`（或用于异步流式传输的`AsyncIterator`），它仅生成一个值：来自底层聊天模型提供者的最终输出。

:::提示

**默认**实现**不**支持逐个令牌的流式传输，但它确保模型可以替换为任何其他模型，因为它支持相同的标准接口。

:::

逐个令牌流式传输的能力取决于提供者是否实现了适当的流式传输支持。

查看哪些[集成支持逐个令牌流式传输](/docs/integrations/chat/)。

## 同步流式传输

下面我们使用`|`来帮助可视化令牌之间的分隔符。

In [ ]:
from langchain_anthropic.chat_models import ChatAnthropic

chat = ChatAnthropic(model="claude-3-haiku-20240307")
for chunk in chat.stream("为我写一首关于月球上的金鱼的单节歌曲"):
    print(chunk.content, end="|", flush=True)

Here| is| a| |1| |verse| song| about| gol|dfish| on| the| moon|:|

Floating| up| in| the| star|ry| night|,|
Fins| a|-|gl|im|mer| in| the| pale| moon|light|.|
Gol|dfish| swimming|,| peaceful| an|d free|,|
Se|ren|ely| |drif|ting| across| the| lunar| sea|.|

## 异步流式传输

In [ ]:
from langchain_anthropic.chat_models import ChatAnthropic

chat = ChatAnthropic(model="claude-3-haiku-20240307")
async for chunk in chat.astream("为我写一首关于月球上的金鱼的单节歌曲"):
    print(chunk.content, end="|", flush=True)

Here| is| a| |1| |verse| song| about| gol|dfish| on| the| moon|:|

Floating| up| above| the| Earth|,|
Gol|dfish| swim| in| alien| m|irth|.|
In| their| bowl| of| lunar| dust|,|
Gl|it|tering| scales| reflect| the| trust|
Of| swimming| free| in| this| new| worl|d,|
Where| their| aqu|atic| dream|'s| unf|ur|le|d.|

## Astream 事件

聊天模型还支持标准的[astream 事件](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.astream_events)方法。

如果您正在从包含多个步骤的更大 LLM 应用程序（例如由提示、LLM 和解析器组成的 LLM 链）中流式传输输出，此方法非常有用。

In [ ]:
from langchain_anthropic.chat_models import ChatAnthropic

chat = ChatAnthropic(model="claude-3-haiku-20240307")
idx = 0

async for event in chat.astream_events(
    "为我写一首关于月球上的金鱼的单节歌曲"
):
    idx += 1
    if idx >= 5:  # 截断输出
        print("...已截断")
        break
    print(event)

{'event': 'on_chat_model_start', 'data': {'input': 'Write me a 1 verse song about goldfish on the moon'}, 'name': 'ChatAnthropic', 'tags': [], 'run_id': '1d430164-52b1-4d00-8c00-b16460f7737e', 'metadata': {'ls_provider': 'anthropic', 'ls_model_name': 'claude-3-haiku-20240307', 'ls_model_type': 'chat', 'ls_temperature': None, 'ls_max_tokens': 1024}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '1d430164-52b1-4d00-8c00-b16460f7737e', 'name': 'ChatAnthropic', 'tags': [], 'metadata': {'ls_provider': 'anthropic', 'ls_model_name': 'claude-3-haiku-20240307', 'ls_model_type': 'chat', 'ls_temperature': None, 'ls_max_tokens': 1024}, 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-1d430164-52b1-4d00-8c00-b16460f7737e', usage_metadata={'input_tokens': 21, 'output_tokens': 2, 'total_tokens': 23, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '1d430164-52